# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np
import pandas as pd

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import matplotlib.pyplot as plt
from matplotlib import cm
from HouseholdSpecializationModel import HouseholdSpecializationModelClass


# Question 1

Assignment: Illustrate the effect of a change in both σ = {0.5,1.0,1.5} and α ∈ {0.25,0.50,0.75} on the HF/HM Ratio.

Our Solution: See comments below

In [ ]:
hsm = HouseholdSpecializationModelClass()

#generate the values for both alpha and sigma
alpha_values = (0.25, 0.5, 0.75)
sigma_values = (0.5, 1.0, 1.5)

df = pd.DataFrame(columns = pd.Index(alpha_values, name="alpha"), index = pd.Index(sigma_values, name = "sigma"))

for i in alpha_values:
    for j in sigma_values:
        hsm.par.alpha = i
        hsm.par.sigma = j
        results = hsm.solve_discrete()
        ratio = results.HF / results.HM
        df.loc[j,i] = f"{ratio:.2f}"   # used to update the value in the DataFrame at the row index "j" and column index "i" with a string representation of the "ratio" value rounded to two decimals.    
        
header = pd.MultiIndex.from_product([['alpha'], alpha_values])
df.columns = header
print(df.to_string())

#create a meshgrid of the above values
alpha, sigma = np.meshgrid(alpha_values, sigma_values)

#generate a 3x3 matrix of zeros and use for loops to calculate the HF/HM ratio for each value of sigma & alpha
#solve the model discretly thereby calculating the ratio for each instance [i,j]
HF_HM_ratio = np.zeros((3, 3))

for i in range (alpha.shape[0]):
    for j in range (sigma.shape[1]):
        hsm.par.alpha = alpha[i,j]
        hsm.par.sigma = sigma[i,j]
        opt = hsm.solve_discrete()
        HF_HM_ratio[i,j] = (opt.HF/opt.HM)

#3D plot to encompass all the effects of variable changes in one illustration
%matplotlib inline
fig = plt.figure() 
ax = fig.add_subplot(1,1,1,projection='3d') 
ax.plot_surface(alpha,sigma,HF_HM_ratio, cmap=cm.Blues, edgecolor='black'); 

#making the plot pretty
ax.set_xlabel('alpha') 
ax.set_ylabel('sigma')
ax.set_zlabel('HF/HM - ratio')
ax.set_title("Plot 1 - Effect of changing alphas and sigmas on HF/HM-ratio")
ax.zaxis.labelpad=0.05
ax.invert_xaxis()
fig.tight_layout()


Economic interpretation: We see from Plot 1 that by increasing either alpha or sigma, we see an increase in the HF/HM ratio. We can see that by holding a small value of sigma constant (i.e. 0.5) and increasing alpha, we get a smaller variation in the HF/HM ratio than if we were to have a large value of sigma (i.e. 1.5).

When we increase household production (σ and α), women will tend to work more from home than men. With very large values of sigma and alpha, we see that the HF/HM ratio will be very high and so women will tend to work a lot more from home than men. 

# Question 2

Assignment: Plot log(HF/HM) against log(wF/wM) for wF ∈[0.8,0.9,1.0,1.1,1.2]

Our Solution: See comments in the code down below

In [ ]:
#reset alpha and sigma to the values specified in the project
hsm.par.alpha = 0.5
hsm.par.sigma = 1

#get wages for male and female from the HouseholdSpecializationModel
wF = hsm.par.wF_vec
wM = hsm.par.wM

#create empty lists for the values of log(HF/HM) and log(wF/wM)
log_ratio = []
log_wages = []

#iterate over the different values for wF and find the optimal values for HF and HM using the discrete solver from the 
#HouseholdSpecializationModel. Then append resulting values to the lists above
for femalewages in wF:
    hsm.par.wF = femalewages
    opt = hsm.solve_discrete()
    log_ratio.append(np.log(opt.HF/opt.HM))
    log_wages.append(np.log(femalewages/wM))

#plot the resulting values for log(HF/HM) and log(wF/wM) against each other
plt.plot(log_wages, log_ratio)
plt.xlabel("log wF/wM")
plt.ylabel("log HF/HM")
plt.title("Plot 2 - Effect of discretly changing wF on log(HF/HM)")
plt.show()

Economic interpretation: The higher the female wage wF gets the lower the value for log(HF/HM). If the female wage wF at the labour market is low the optimal solution of our problem would be to have the female work more hours in the household compared to the man. If wages for females are higher in the labour market the balance tips in favour of the man doing more work at home 

# Question 3

Assignment: Plot log HF/HM against log wF/wM for wF ∈[0.8,0.9,1.0,1.1,1.2] and solve continuously


Our Solution: To solve the model continuously we must define the solve() function in the HouseholdSpecializationModel.py file. We define an objective function and call it the value_of_choice whereby we list each of the variables LM, HM, LF, HF (in order) and set it equal to x, to which we then return the negative of the utility function. Additionally, we must also set constraints on our variables to restrict our set of solutions to be within (0,24) given the layout of the model parameters. We include an arbitrary initial guess that we then include in our optimize.minimize command that is used to minimize the negative of our value_of_choice function subject to our set constraints and bounds. The approach uses the Nelder-Mead method in order to solve the model in this multi-dimensional space. 

In [ ]:
#get wages for male and female from the householdspecializationmodel
wF = hsm.par.wF_vec
wM = hsm.par.wM

#reset alpha and sigma values to a constant term
hsm.par.alpha = 0.5
hsm.par.sigma = 1

#create empty lists for the values of log(HF/HM) and log(wF/wM)
log_wages_continuous = []
log_ratio_continuous = []

#iterate over the different values for wF and find the optimal values for HF and HM using the discrete solver from the 
#HouseholdSpecializationModel. Then append resulting values to the lists above
for femalewages in wF:
    hsm.par.wF = femalewages
    opt = hsm.solve()
    log_ratio_continuous.append(np.log(opt.HF/opt.HM))
    log_wages_continuous.append(np.log(femalewages/wM))

#plot the resulting values for log(HF/HM) and log(wF/wM) against each other
plt.plot(log_wages_continuous, log_ratio_continuous)
plt.xlabel("log wF/wM")
plt.ylabel("log HF/HM")
plt.title("Plot 3 - Effect of continuous changing wF on log(HF/HM) ")
plt.show()

Economic interpretation (same as for question 2): The higher the female wage wF gets the lower the value for log(HF/HM). If the female wage wF at the labour market is low the optimal solution of our problem would be to have the female work more hours in the household compared to the man. If wages for females are higher in the labour market the balance tips in favour of the man doing more work at home

# Question 4

Assignment: 

Choose α and σ such it minimizes (β0 − β0)^2 + (β1 − β1)^2. 

We illustrate our results of log HF/HM against log WF/WM for Wf ∈[0.8,0.9,1.0,1.1,1.2]. We use the continuous values for Lm, Lf, Wm, Wf.


Our Solution:

We define our objective function within our estimate and minimize it to find the optimal values of alpha and sigma that best fit the model. Our objective function is minimized with respect to the regression and vector of female wages as can be seen in the HouseholdSpecializationModel.py. By calling on the solver, we obtain optimal values for our fitted parameters and display the results below:

In [ ]:
hsm.printing_estimate()


Economic interpretation:

The regression equation log HF/HM = β0 + β1 log wF/wM showcases the relationship between the log of the ratio of hours worked at home by females versus males as the dependent variable and the log of their relative wages as the independent variable. 

The value of β1 = -0.1 represents the effect a one unit change in the wage ratio has on the ratio of hours worked at home. Essentially, the model states that a 1% increase in the ratio of female wages in relation to male wages represents a -0.1 decrease in the amount of hours that females spend working at home. This indicates that as the ratio of female to male wages increases, the amount of time spent working at home by females decreases. 

Our σ = 0.99 indicates a consumption of home production represented by the Cobb-Douglas function as sigma is approximately 1.0. The value of α = 0.80 indicates that household production is not equally productive amongst males and females and as a result females tend to be more productive at home. 


# Question 5

Assignment: Suggest and implement an extension of the model, and analyze whether or not it can help match the data when α = 0.5.


Our Solution:

In [ ]:
hsm.printing_estimate_alpha()

Economic interpretation:

ADD CONCISE CONLUSION TO THE WHOLE MODEL.